# Project Mcnulty:

### Starting to explore the data

In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
loan_df = pd.read_csv("data/loan.csv")

/Users/williamcosby/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 74 columns):
id                             887379 non-null int64
member_id                      887379 non-null int64
loan_amnt                      887379 non-null float64
funded_amnt                    887379 non-null float64
funded_amnt_inv                887379 non-null float64
term                           887379 non-null object
int_rate                       887379 non-null float64
installment                    887379 non-null float64
grade                          887379 non-null object
sub_grade                      887379 non-null object
emp_title                      835922 non-null object
emp_length                     887379 non-null object
home_ownership                 887379 non-null object
annual_inc                     887375 non-null float64
verification_status            887379 non-null object
issue_d                        887379 non-null object
loan_status          

# Using SQLite

In [9]:
con = sqlite3.connect('data/database.sqlite')

In [10]:
loans = pd.read_sql_query("select * from loan;", con)
print(loans.head())

   index       id  member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
0      0  1077501  1296599.0     5000.0       5000.0           4975.0   
1      1  1077430  1314167.0     2500.0       2500.0           2500.0   
2      2  1077175  1313524.0     2400.0       2400.0           2400.0   
3      3  1076863  1277178.0    10000.0      10000.0          10000.0   
4      4  1075358  1311748.0     3000.0       3000.0           3000.0   

         term int_rate  installment grade     ...      total_bal_il il_util  \
0   36 months   10.65%       162.87     B     ...               NaN     NaN   
1   60 months   15.27%        59.83     C     ...               NaN     NaN   
2   36 months   15.96%        84.33     C     ...               NaN     NaN   
3   36 months   13.49%       339.31     C     ...               NaN     NaN   
4   60 months   12.69%        67.79     B     ...               NaN     NaN   

  open_rv_12m open_rv_24m  max_bal_bc all_util total_rev_hi_lim inq_fi  \
0         Na

In [11]:
loans.columns

Index([u'index', u'id', u'member_id', u'loan_amnt', u'funded_amnt',
       u'funded_amnt_inv', u'term', u'int_rate', u'installment', u'grade',
       u'sub_grade', u'emp_title', u'emp_length', u'home_ownership',
       u'annual_inc', u'verification_status', u'issue_d', u'loan_status',
       u'pymnt_plan', u'url', u'desc', u'purpose', u'title', u'zip_code',
       u'addr_state', u'dti', u'delinq_2yrs', u'earliest_cr_line',
       u'inq_last_6mths', u'mths_since_last_delinq', u'mths_since_last_record',
       u'open_acc', u'pub_rec', u'revol_bal', u'revol_util', u'total_acc',
       u'initial_list_status', u'out_prncp', u'out_prncp_inv', u'total_pymnt',
       u'total_pymnt_inv', u'total_rec_prncp', u'total_rec_int',
       u'total_rec_late_fee', u'recoveries', u'collection_recovery_fee',
       u'last_pymnt_d', u'last_pymnt_amnt', u'next_pymnt_d',
       u'last_credit_pull_d', u'collections_12_mths_ex_med',
       u'mths_since_last_major_derog', u'policy_code', u'application_type',
   

# Drop columns with lots of missing variables

In [12]:
columns = []
for column in loans.columns:
    if loans[column].count() > 800000:
        columns.append(column)
loans = loans[columns]

In [13]:
len(loans.columns)

53

In [21]:
loans['total_rec_int'].head(10)

0     861.07
1     435.17
2     603.65
3    2209.33
4    1009.07
5     631.38
6    3025.99
7     938.14
8     294.94
9     533.42
Name: total_rec_int, dtype: float64

# classify loans

## loan_status gives current status of the loan:

### Values are:
* Current
* Fully Paid
* Late (16-30 days)
* Late (31-120 days)
* In Grace Period
* Charged Off
* Default
* nan


In [119]:
loans['loan_status'].unique()
loans['loan_status'].replace(['Fully Paid','Current',
                              'Does not meet the credit policy. Status:Fully Paid'], 
                             value=1,inplace=True)

loans['loan_status'].replace(['Charged Off', 'In Grace Period','Default','Late (31-120 days)','Late (16-30 days)',
                             'Does not meet the credit policy. Status:Charged Off'], 
                             value=0,inplace='True')

loans = loans[(loans['loan_status'] == 0) | (loans['loan_status'] == 1)]
# loans = loans[loans['loan_status'] != 'Issued']

In [138]:
loans['loan_status'] = pd.to_numeric(loans['loan_status'])

# Build an initial model (somehow)

In [139]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
loans_numeric = loans.select_dtypes(include=numerics)

In [147]:
loans_numeric.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808646 entries, 42536 to 887382
Data columns (total 31 columns):
index                         808646 non-null int64
member_id                     808646 non-null float64
loan_amnt                     808646 non-null float64
funded_amnt                   808646 non-null float64
funded_amnt_inv               808646 non-null float64
installment                   808646 non-null float64
annual_inc                    808646 non-null float64
loan_status                   808646 non-null int64
dti                           808646 non-null float64
delinq_2yrs                   808646 non-null float64
inq_last_6mths                808646 non-null float64
open_acc                      808646 non-null float64
pub_rec                       808646 non-null float64
revol_bal                     808646 non-null float64
total_acc                     808646 non-null float64
out_prncp                     808646 non-null float64
out_prncp_inv            

## Drop all with missing values

In [141]:
loans_numeric.dropna(inplace=True)

/Users/williamcosby/anaconda/envs/data/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [142]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [148]:
y = loans_numeric['loan_status']

In [144]:
# loans_x = loans_numeric.drop(['loan_status'],axis=1)

In [162]:
X = loans_numeric

In [163]:
X.columns

Index([u'index', u'member_id', u'loan_amnt', u'funded_amnt',
       u'funded_amnt_inv', u'installment', u'annual_inc', u'dti',
       u'delinq_2yrs', u'inq_last_6mths', u'open_acc', u'pub_rec',
       u'revol_bal', u'total_acc', u'out_prncp', u'out_prncp_inv',
       u'total_pymnt', u'total_pymnt_inv', u'total_rec_prncp',
       u'total_rec_int', u'total_rec_late_fee', u'recoveries',
       u'collection_recovery_fee', u'last_pymnt_amnt',
       u'collections_12_mths_ex_med', u'policy_code', u'acc_now_delinq',
       u'tot_coll_amt', u'tot_cur_bal', u'total_rev_hi_lim'],
      dtype='object')

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [165]:
X_train.head()

,index,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,...,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
562586,96298,67399945.0,28000.0,28000.0,27975.0,992.02,75000.0,27.43,2.0,0.0,...,0.0,0.0,0.0,992.02,0.0,1.0,0.0,0.0,524644.0,33400.0
744710,278422,52931852.0,16000.0,16000.0,16000.0,495.94,120000.0,17.12,0.0,1.0,...,0.0,0.0,0.0,495.94,0.0,1.0,0.0,0.0,220962.0,54400.0
573630,107342,64688118.0,12000.0,12000.0,12000.0,392.81,80000.0,31.03,3.0,0.0,...,0.0,0.0,0.0,392.81,0.0,1.0,1.0,0.0,85012.0,45000.0
47542,5006,11507002.0,30000.0,30000.0,30000.0,952.60,240000.0,13.05,0.0,1.0,...,0.0,0.0,0.0,952.60,0.0,1.0,0.0,0.0,755822.0,52900.0
125031,82495,4381597.0,33425.0,33425.0,33425.0,835.56,75000.0,22.24,0.0,0.0,...,0.0,0.0,0.0,835.56,0.0,1.0,0.0,0.0,143644.0,63500.0


In [166]:
random_forest = RandomForestClassifier(n_estimators=500,random_state=42, max_depth=10)
random_forest = random_forest.fit(X_train, y_train)

In [167]:
random_forest.score(X_test,y_test)

0.97339587953535534

In [172]:
sorted(zip(X.columns,random_forest.feature_importances_), key=lambda tup: tup[1], reverse=True)

[('recoveries', 0.19229317668207396),
 ('collection_recovery_fee', 0.14149982761996219),
 ('total_rec_prncp', 0.11477032851975925),
 ('last_pymnt_amnt', 0.09176722376090321),
 ('out_prncp', 0.077945827283388808),
 ('out_prncp_inv', 0.077025118645434279),
 ('total_pymnt_inv', 0.049966214461567453),
 ('total_pymnt', 0.044204368792712077),
 ('loan_amnt', 0.033027264563106798),
 ('funded_amnt_inv', 0.032596412583618777),
 ('funded_amnt', 0.032331266578402677),
 ('installment', 0.031511495529935214),
 ('total_rec_late_fee', 0.028736634896624771),
 ('member_id', 0.024356400686138785),
 ('total_rec_int', 0.01162065608316378),
 ('index', 0.0071169481225173059),
 ('revol_bal', 0.0016092965404912137),
 ('dti', 0.0014318129465704745),
 ('total_rev_hi_lim', 0.0014296157797533101),
 ('annual_inc', 0.001326780560464841),
 ('inq_last_6mths', 0.00097406148917583636),
 ('tot_cur_bal', 0.00092044323725111541),
 ('total_acc', 0.00050545986685998981),
 ('open_acc', 0.00045528616828124468),
 ('delinq_2yrs'